## Generate synthetic data [needs work in course selection algortihm]
## convert text to numeric using tfidf [not used currently]

In [34]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
import re
import nlp
#https://chat.openai.com/share/c141bbaf-9fa9-41df-b29a-68653cfafd48

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df_c_cleaned = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")
df_c_cleaned=df_c_cleaned.rename(columns={"Unnamed: 0":"course_number"})
df_j_cleaned=pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_salary.csv")
df_s_skills_list = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/skills_dataset/skills_list_df.csv")

skills_list = df_s_skills_list["skills_list"].unique().tolist()

In [36]:
#filtering unmatched rows
df_j_cleaned = df_j_cleaned[df_j_cleaned["title_cleaned"]!="unmatched"]
#df_j_cleaned[df_j_cleaned["title_cleaned"]=="unmatched"].to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_title_unmatched.csv")


In [37]:
# Loading the JSON file back into a dictionary
import json
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'r') as json_file:
    sorted_full_job_progression_dict_lower = json.load(json_file)

print(sorted_full_job_progression_dict_lower)
# Output: {'name': 'John', 'age': 30, 'city': 'New York'}


{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'], 'ai developer': ['senior ai developer', 'ai architect', 'ai project manager', 'head of ai', 'chief ai officer'], 'ai project manager': ['head of ai', 'chief ai officer'], 'big data architect': ['head of big data', 'director of data engineering', 'chief information officer'], 'big data engineer': ['senior big data engineer', 'big data architect', 'head of big data', 'director of data engineering', 'chief information officer'], 'business analysis manager': ['business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business analyst': ['senior business analyst', 'business analysis manager', 'business intelligence analyst', 'director of business analysis', 'chief strategy officer', 'data analyst', 'data scientist'], 'business intelligence analyst': ['senior business intelligence analyst', 'business intelligence manager', 'director of business intel

In [38]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
# Assuming you have the following data loaded into pandas DataFrames
# jobs_df = pd.DataFrame({'Job': ['Data Analyst', 'Data Scientist', ...], 'Skills': [['SQL', 'Python'], ['Machine Learning', 'Python', 'SQL'], ...]})
# courses_df = pd.DataFrame({'Course': ['Intro to Python', 'Advanced Machine Learning', ...], 'Skills_Taught': [['Python'], ['Machine Learning', 'Deep Learning'], ...]})
# skills_df = pd.DataFrame({'Skill': ['Python', 'SQL', 'Machine Learning', ...]})
# A simple function to select a random job and its required skills


def select_random_job(jobs_df):
    random_idx = random.randint(0, len(jobs_df) - 1)
    random_job = jobs_df.iloc[random_idx]
    return random_job.title_cleaned, random_job.skills_tagged,random_job.normalized_min,random_job.normalized_max


def select_next_job(current_job, jobs_df, set_job_unique,job_progression_dict):
    next_job,next_skills,random_job_selection_flag="","",0
    # If the current job is in the progression dict, select the next job from there
    if current_job in job_progression_dict:
        next_jobs = job_progression_dict[current_job]
        #next_jobs=list(set_job_unique.intersection(set(next_jobs)))
        if len(next_jobs)!=0:
            next_job_flag = True
            i=0
            while next_job_flag and i<len(next_jobs):
                if next_jobs[i] in set_job_unique:
                    next_job = next_jobs[i]
                    next_job_flag=False
                i+=1
            if next_job_flag==False:
                #next_job = random.choice(next_jobs)
                #print(next_job, " choosen")
                try:
                    #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                    next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                    if not next_skills_rows.empty:
                        random_row = next_skills_rows.sample(n=1)
                        next_skills = random_row['skills_tagged'].values[0]
                except:
                    next_job = random.choice(next_jobs)
                    #next_skills = jobs_df[jobs_df['title_cleaned'] == next_job]['skills_tagged'].values[0]
                    next_skills_rows = jobs_df[jobs_df['title_cleaned'] == next_job]
                    if not next_skills_rows.empty:
                        random_row = next_skills_rows.sample(n=1)
                        next_skills = random_row['skills_tagged'].values[0]
            else:
                # If not, fall back to random selection for this example
                next_job, next_skills,_,_ = select_random_job(jobs_df)
                random_job_selection_flag=1
        else:
            next_job, next_skills,_,_ = select_random_job(jobs_df)
            random_job_selection_flag=1

    else:
        next_job, next_skills,_,_ = select_random_job(jobs_df)
        random_job_selection_flag=1
    return next_job, next_skills,random_job_selection_flag





# A simple function to select a course that teaches at least one of the required skills
def select_relevant_course(courses_df, required_skills):
    relevant_courses = courses_df[courses_df['Skills_Taught'].apply(lambda x: any(skill in x for skill in required_skills))]
    if not relevant_courses.empty:
        return relevant_courses.sample()
    return None


def calculate_course_score(course, skill_gap, weight_skill_match=1, weight_subscribers=0.001, weight_rating=0.5):
    # Calculate the proportion of skill gap covered by course
    skills_matched = sum(skill in course['skills_tagged'] for skill in skill_gap)
    skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0

    # Factor in course popularity and rating
    #subscriber_score = course['num_subscribers'] * weight_subscribers if 'num_subscribers' in course else 0
    #rating_score = course['rating'] * weight_rating if 'rating' in course else 0

    # Calculate total score
    total_score = (skill_match_score * weight_skill_match) #+ subscriber_score + rating_score
    return total_score

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_c_cleaned["full_description"])

def calculate_similarity_score(skill_gap,df_c_cleaned):

    def preprocess_text(text):
        return " ".join(token.lemma_.lower() for token in nlp(text) if not token.is_punct and not token.is_stop)

    skill_gap_text = " ".join(skill_gap)
    # Vectorize the text

    skill_gap_vec = tfidf_vectorizer.transform([skill_gap_text])
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(skill_gap_vec, tfidf_matrix).flatten()
    df_c_cleaned["cosine_score"]=cosine_similarities
    return df_c_cleaned

def select_relevant_course(df_c_cleaned, skill_gap):
    # Apply the scoring function to each course
    df_c_cleaned['course_score'] = df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)
    df_c_cleaned = calculate_similarity_score(skill_gap,df_c_cleaned)
    # Sort courses based on the calculated score
    #df_c_cleaned = df_c_cleaned.sort_values(by='course_score', ascending=False)
    df_c_cleaned = df_c_cleaned.sort_values(by='cosine_score', ascending=False)
    # Select the top course if available
    top_course = df_c_cleaned.iloc[0] if not df_c_cleaned.empty else None
    top_10_course_number = df_c_cleaned["course_number"].iloc[0:10].tolist()
    top_10_course_number_score = {x:round(y,3) for x,y in zip(df_c_cleaned["course_number"].iloc[0:10].tolist(),df_c_cleaned["cosine_score"].iloc[0:10].tolist())}
    return top_course,top_10_course_number_score,top_10_course_number

# Select the most relevant course based on the skill gap
#relevant_course = select_relevant_course(df_c_cleaned, skill_gap)
#relevant_course.head()

# Generate synthetic user data
def generate_synthetic_data(jobs_df, courses_df, skills_df, num_samples=10):
    synthetic_data = []
    set_job_unique = set(jobs_df["title_cleaned"].unique())
    for _ in range(num_samples):
        #select_random_job(jobs_df)
        current_job, current_skills,current_min_salary,current_max_salary = select_random_job(jobs_df)
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,set_job_unique,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        current_skills=ast.literal_eval(current_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        if  len(skill_gap)==0:
            skill_gap = list(next_skills)
        # Find a course that teaches one of the skills in the gap
        course,top_10_courses_score,top_10_courses_number = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"

        if len(current_skills) and len(next_skills):
            # Append to the synthetic data list
            synthetic_data.append({
                'Current Job': current_job,
                'Next Job': next_job,
                'Current Skills': current_skills,
                'Next Skills': next_skills,
                'Skill Gap': skill_gap,
                'Current Min Salary':current_min_salary,
                'Current Max Salary':current_max_salary,
                'course title':course_title,
                'Course Taken': course_name,
                'Course Skills': course_skills,
                'top_10_course_numbers':top_10_courses_number,
                'top_10_course_score':top_10_courses_score,
                'random next job':random_job_selection_flag
            })
    
    
    return pd.DataFrame(synthetic_data)


# Generate synthetic user data
def generate_synthetic_next_job_data(jobs_df,courses_df,synthetic_user_data):
    synthetic_data = []
    set_job_unique = set(jobs_df["title_cleaned"].unique())
    for i,iterrow in synthetic_user_data.iterrows():
        #print(i, iterrow["Next Job"],iterrow["Next Skills"])
        #select_random_job(jobs_df)
        current_job, current_skills,current_min_salary,current_max_salary = iterrow["Next Job"],(iterrow["Next Skills"]),iterrow["Current Min Salary"],iterrow["Current Max Salary"]
        # Select a job to progress to that requires at least one additional skill
        next_job, next_skills,random_job_selection_flag = select_next_job(current_job,jobs_df,set_job_unique,sorted_full_job_progression_dict_lower)
        #while set(next_skills).issubset(current_skills):
        #    next_job, next_skills = select_random_job(jobs_df)
        next_skills=ast.literal_eval(next_skills)
        # Find the skill gap
        skill_gap = list(set(next_skills) - set(current_skills))
        #print(skill_gap)
        # Find a course that teaches one of the skills in the gap
        course,top_10_courses_score,top_10_courses_number = select_relevant_course(courses_df, skill_gap)
        #print(course)
        if course is not None:
            course_name = course.course_title
            course_skills = course.skills_tagged
            course_title = course.title
        else:
            # If no single course covers the gap, select a random one for this synthetic example
            course_name = "Random Course"
            course_skills = "Random Skill"
        
        if len(current_skills) and len(next_skills):
            # Append to the synthetic data list
            synthetic_data.append({
                'Current Job': current_job,
                'Next Job': next_job,
                'Current Skills': current_skills,
                'Next Skills': next_skills,
                'Skill Gap': skill_gap,
                'Current Min Salary':current_min_salary,
                'Current Max Salary':current_max_salary,
                'course title':course_title,
                'Course Taken': course_name,
                'Course Skills': course_skills,
                'top_10_course_numbers':top_10_courses_number,
                'top_10_course_score':top_10_courses_score,
                'random next job':random_job_selection_flag
            })
    
    return pd.DataFrame(synthetic_data)



# Generate the synthetic data
synthetic_user_data_v1 = generate_synthetic_data(df_j_cleaned, df_c_cleaned, skills_list,10000)
synthetic_user__next_data_v2 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user_data_v1)
#synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2],axis=0)
synthetic_user__next_data_v3 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v2)
synthetic_user__next_data_v4 = generate_synthetic_next_job_data(df_j_cleaned, df_c_cleaned,synthetic_user__next_data_v3)
#synthetic_user_data = pd.concat([synthetic_user__next_data_v3,synthetic_user__next_data_v4],axis=0)
synthetic_user_data = pd.concat([synthetic_user_data_v1,synthetic_user__next_data_v2,synthetic_user__next_data_v3,synthetic_user__next_data_v4],axis=0)
#print(synthetic_user_data.head())


In [67]:
df_j_cleaned[50:100]

,Unnamed: 0.1,Unnamed: 0,company,truncatedCompany,companyRating,companyReviewCount,searched_title,title,title_cleaned,normTitle,min,max,type,snippet,jobDescription,skills_tagged,normalized_min,normalized_max
176,176,176,DataDrive,DataDrive,NaN,NaN,Data Architect,tableau architect,enterprise architect,Enterprise Architect,100000.000000,110000.000000,yearly,Including the ability to communicate data stru...,role overview tableau architect dynamic enviro...,"{'business reporting', 'data strategy', 'data ...",-0.194360,-0.197737
177,177,177,Visions in View Inc,Visions in View Inc,3.1,7.0,Data Architect,data architect,data architect,Data Warehouse Architect,124800.000000,145600.000000,yearly,Extensive experience on data engineering and m...,job description possess years data architectin...,"{'data migration', 'database systems', 'big da...",0.319459,0.533546
178,178,178,Capgemini,Capgemini,3.7,9764.0,Data Architect,data architect,data architect,Data Warehouse Architect,99712.000000,109712.000000,yearly,Work with multi-functional teams to implement ...,job description job title data architect locat...,"{'business requirements', 'http', 'postgresql'...",-0.200327,-0.203653
179,179,179,Delviom LLC,Delviom LLC,4.5,6.0,Data Architect,tableau architect,enterprise architect,Enterprise Architect,145600.000000,166400.000000,yearly,Develop and implement the initial Tableau User...,position tableau architect location remote dur...,set(),0.750405,0.960813
180,180,180,TASC Technical Services LLC,TASC Technical Services LLC,4.3,7.0,Data Architect,data center administrator,database administrator,Data Administrator,124800.000000,145600.000000,yearly,* Collaborate with cross-functional teams to p...,northrop grumman space systems linthicum md se...,"{'bitbucket', 'data migration', 'docker', 'dat...",0.319459,0.533546
182,182,182,MCKESSON,MCKESSON,3.5,7170.0,Data Architect,sap hana db architect,enterprise architect,Enterprise Architect,140400.000000,150400.000000,yearly,Mckesson Corporation is looking for a SAP HANA...,mckesson impact driven fortune company touches...,"{'business requirements', 'agile methodology',...",0.642668,0.632146
183,183,183,NAVEX Global,NAVEX Global,3.0,167.0,Data Architect,senior enterprise architect,enterprise architect,Senior Enterprise Architect,160000.000000,170000.000000,yearly,"Knowledge of the following technologies: AWS, ...",fun work company people truly believe committe...,"{'docker', 'application services', 'nosql', 'p...",1.048751,1.034763
184,184,184,Shure,Shure,4.0,91.0,Data Architect,architect sap business applications supply chain,ai architect,IT Architect,105464.943636,114102.670909,yearly,Ensure solution design proposal adhere to the ...,overview shure embarked multi year multi phase...,"{'business requirements', 'business intelligen...",-0.081135,-0.113461
186,186,186,GreenState Credit Union,GreenState Credit Union,3.3,45.0,Data Architect,data architect,data architect,IT Architect,119078.700000,129078.700000,yearly,Provide recommendations and take appropriate a...,greenstate credit union data architect ia nort...,"{'business requirements', 'data validation', '...",0.200922,0.194172
187,187,187,GM Financial,GM Financial,3.6,938.0,Data Architect,lead data architect,senior data architect,Lead Data Architect,112200.000000,122200.000000,yearly,Lead team in activities related to the data po...,overview gm financial gm financial wholly owne...,"{'data management', 'databricks', 'data migrat...",0.058406,0.052872


In [68]:
synthetic_user_data

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,Current Min Salary,Current Max Salary,course title,Course Taken,Course Skills,top_10_course_numbers,top_10_course_score,random next job
0,director of product management,chief product officer,{c},"{problem solving, application design, analytic...","[problem solving, application design, analytic...",1.250301,1.234592,complete data science project management course,data mining,"{'data mining', 'data science', 'project manag...","[615, 974, 1542, 968, 2953, 323, 1437, 1240, 2...","{615: 0.311, 974: 0.28, 1542: 0.266, 968: 0.26...",0
1,business analyst,senior business analyst,"{java, sql}","{product lifecycle management, technology solu...","[product lifecycle management, technology solu...",-0.542432,-0.320986,avaya ip office server,artificial intelligence,{'artificial intelligence'},"[701, 1542, 1745, 1992, 1434, 1182, 1471, 1124...","{701: 0.212, 1542: 0.165, 1745: 0.165, 1992: 0...",0
2,data product manager,senior data product manager,"{analytics, project management, business analy...","{quantitative analysis, sql, machine learning,...","[sql, quantitative analysis, computer science,...",-0.413977,-0.415478,data analyst python beginners,undefined,"{'data visualization', 'python programming lan...","[1670, 1478, 1520, 2036, 892, 904, 788, 1763, ...","{1670: 0.354, 1478: 0.329, 1520: 0.295, 2036: ...",0
3,senior data analyst,data engineer,"{bigquery, python, sql, airflow, problem solvi...","{data processing, sql, apache kafka, data ware...","[data processing, data warehousing, computer s...",0.116418,0.110388,f rdan apache kafka z kurulum kullan java python,apache kafka,"{'apache kafka', 'java', 'python'}","[2563, 2135, 2525, 323, 2036, 1991, 1542, 1520...","{2563: 0.409, 2135: 0.285, 2525: 0.264, 323: 0...",0
4,database administrator,senior database administrator,"{data modeling, sql, data quality, pl sql, jav...","{technology solutions, sql, computer science, ...","[technology solutions, computer science, datab...",-1.320774,-1.404756,postgresql database administration masterclass,database administration,"{'database management systems', 'database admi...","[1783, 1233, 109, 49, 1124, 1292, 1592, 1998, ...","{1783: 0.584, 1233: 0.206, 109: 0.185, 49: 0.1...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,chief product officer,big data engineer,"{product lifecycle management, analytics, data...","{data ingestion, data transformation, data pip...","[data processing, data ingestion, data warehou...",0.132992,0.126822,apache spark hands specialization big data ana...,apache spark,"{'big data analytics', 'data science', 'big da...","[303, 1991, 1503, 1728, 1634, 1722, 904, 927, ...","{303: 0.425, 1991: 0.4, 1503: 0.391, 1728: 0.3...",0
8056,data architect,data manager,"{data warehousing, computer science, data arch...","{technology solutions, python, computer scienc...","[technology solutions, data analysis, data man...",0.609519,0.599280,getting started data management,database management,"{'data integration', 'big data', 'python progr...","[323, 1542, 1670, 446, 755, 444, 1645, 1091, 1...","{323: 0.401, 1542: 0.366, 1670: 0.345, 446: 0....",0
8057,machine learning manager,chief technology officer,"{llm, data pipelines, artificial intelligence,...","{computer science, analytics, machine learning...",[big data],0.380981,0.349892,big data programming languages big data vs dat...,big data,"{'data science', 'big data'}","[289, 841, 1043, 1119, 1690, 1526, 680, 1520, ...","{289: 0.796, 841: 0.66, 1043: 0.64, 1119: 0.50...",1
8058,data science manager,director of data science,"{cloud computing, sql, python, problem solving...","{sql, ad hoc analysis, data quality, data gove...","[ad hoc analysis, data quality, data governanc...",0.634381,0.623930,predictive analytics model term deposit invest...,undefined,"{'visualization', 'data analysis', 'statistica...","[1957, 1922, 1971, 1861, 323, 961, 1395, 792, ...","{1957: 0.342, 1922: 0.331, 1971: 0.304, 1861: ...",0


In [40]:

synthetic_user_data.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_half_cleaned_v3.csv")



In [43]:
import ast 
df = synthetic_user_data.copy()
# Fill missing values if necessary
df.fillna({'Course Taken': 'no course','course title': 'no course'}, inplace=True)
df["Course Taken"]=df["Course Taken"].apply(lambda x:x.replace("NAN","No Course"))

df["Course Skills"]=df["Course Skills"].apply(lambda x:ast.literal_eval(x))
df["Course Skills"].apply(lambda x: x if len(x)!=0 else "no skills")
#df["Course title"]=df["Course title"].apply(lambda x:x.replace("NAN","No Course"))
def set_to_string(skill_set):
    if not skill_set:  # Checks if the set is empty
        return 'no skills' 
    # Check if skill_set is a string that needs to be evaluated
    if isinstance(skill_set, str):
        try:
            # Try to evaluate the string as a set
            skill_set = ast.literal_eval(skill_set)
        except (ValueError, SyntaxError):
            # Handle cases where the string is not a valid set
            pass  # You might want to return a default value or handle this case as needed
    # Convert to string if it's a set or list
    if isinstance(skill_set, (set, list)):
        return ', '.join(skill_set)
    return skill_set 

set_columns = ['Current Skills', 'Next Skills', 'Skill Gap', 'Course Skills']
for col in set_columns:
    print(col)
    #df[col]=df[col].apply(lambda x:ast.literal_eval(x))
    df[col] = df[col].apply(set_to_string)





Current Skills
Next Skills
Skill Gap
Course Skills


In [44]:
#df.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/synthetic_data_cleaned_v5.csv")


In [64]:
df[['Current Job', 'Next Job', 'Current Skills','Next Skills']].drop_duplicates()

,Current Job,Next Job,Current Skills,Next Skills
0,director of product management,chief product officer,c,"problem solving, application design, analytics..."
1,business analyst,senior business analyst,"java, sql","product lifecycle management, technology solut..."
2,data product manager,senior data product manager,"analytics, project management, business analys...","quantitative analysis, sql, machine learning, ..."
3,senior data analyst,data engineer,"bigquery, python, sql, airflow, problem solvin...","data processing, sql, apache kafka, data wareh..."
4,database administrator,senior database administrator,"data modeling, sql, data quality, pl sql, java...","technology solutions, sql, computer science, d..."
...,...,...,...,...
8045,senior data analyst,data engineer,"data analysis, business requirements, acceptan...","data processing, data modeling, data infrastru..."
8046,data manager,director of analytics,"test data, problem solving, data quality, data...","statistical modeling, machine learning, applic..."
8050,data architect,data manager,"sql, business requirements, power bi, data mod...","data management plan, data capture, applicatio..."
8051,data architect,data manager,"data warehousing, computer science, data archi...","data management, project management, statistic..."


In [62]:
df_j_cleaned[["title_cleaned","skills_tagged"]]

,title_cleaned,skills_tagged
17,chief information officer,"{'data management', 'technology solutions', 'p..."
22,chief information officer,"{'data driven decision making', 'http'}"
24,chief information officer,"{'program analysis', 'statistical methods', 'e..."
33,chief strategy officer,"{'business development', 'data analysis', 'pro..."
36,chief technology officer,set()
...,...,...
7611,chief technology officer,"{'project management', 'computer science'}"
7613,chief technology officer,"{'devops', 'cloud operations', 'project manage..."
7616,chief technology officer,{'cloud infrastructure'}
7617,chief technology officer,"{'deep learning', 'cloud computing', 'natural ..."


In [14]:
# convert text to numeric using tfidf [not used currently]

import spacy

# Load SpaCy model
nlp = spacy.load('en_core_web_md')

def preprocess_text(text):
    doc = nlp(text)
    return " ".join(token.lower_ for token in doc if not token.is_punct and not token.is_stop)

text_columns = ['Current Job', 'Next Job', 'Current Skills', 'Next Skills', 'Skill Gap', 'course title', 'Course Skills','Course Taken']
for col in text_columns:
    print(col)
    #df[col] = df[col].apply(preprocess_text)

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming df is your DataFrame and text_columns are the columns to vectorize
tfidf_vectorizer = TfidfVectorizer()

# Create a DataFrame to hold all TF-IDF features
tfidf_df = pd.DataFrame()

for col in ["course title"]:
    # Apply TF-IDF transformation
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[col])

    # Convert to dense format and create a DataFrame
    tfidf_dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

    # Prefix column names with the original column name for clarity
    tfidf_dense.columns = [f"{col}_{feature}" for feature in tfidf_dense.columns]

    # Concatenate with the main TF-IDF DataFrame
    tfidf_df = pd.concat([tfidf_df, tfidf_dense], axis=1)

# Combine the original DataFrame with the TF-IDF features DataFrame
final_df = pd.concat([df, tfidf_df], axis=1)

Current Job
Next Job
Current Skills
Next Skills
Skill Gap
course title
Course Skills
Course Taken


In [15]:
final_df

,Current Job,Next Job,Current Skills,Next Skills,Skill Gap,course title,Course Taken,Course Skills,random next job,course title_ai,course title_bonus,course title_chatgpt,course title_learning,course title_machine,course title_python
0,ai project manager,data scientist,"project management, problem solving, microsoft...","analytics, sql, python, algorithms, data stora...","d, h, u, q, y",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
1,product analyst,data product manager,,"project management, analytics, agile software ...","w, ', v, o, }, {, , g, b, c, a, j, m, ,, p, d...",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
2,data analyst,director of data engineering,"microsoft office, data synchronization, data m...","computer science, test data management, devops","u, v",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
3,unmatched,database administrator,"problem solving, data collection","database administration, sql",q,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
4,senior of data science,statistician,"data analysis, analytics, business intelligenc...","data analysis, statistical analysis, data mana...",no skills,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
5,unmatched,machine learning engineer,"problem solving, project management, data mana...","data mining, bigtable, mapreduce, python, comp...","h, u, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
6,director of product management,chief product officer,"technology solutions, problem solving, risk an...","analytics, data driven decision making, produc...","d, f",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
7,business analyst,business intelligence analyst,"business analysis, qualitative analysis, sql","analytics, business intelligence","g, c",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
8,business intelligence manager,director of business intelligence,"data analysis, analytics, problem solving, bus...","data analysis, analytics, business intelligenc...","k, f, w",machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",0,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
9,director of analytics,unmatched,"analytics, data science, computer science, com...","data structures, computer science, data integr...",k,machine learning z ai python r chatgpt bonus,data science,"logistic regression, multiple linear regressio...",1,0.408248,0.408248,0.408248,0.408248,0.408248,0.408248
